In [5]:

from __future__ import annotations
import os, glob, warnings
from typing import Dict, Any, Optional, List
import numpy as np
import pandas as pd

# Audio / DSP
import librosa
import librosa.display
from scipy.signal import find_peaks


import pandas as pd
import wave 
import os

In [6]:
labels = pd.read_csv('data/raw/training_label.csv')

In [7]:
# ==============================
# Audio features wrapper (wav)
# ==============================
from __future__ import annotations
import os, glob, warnings
from typing import Dict, Any, Optional, List
import numpy as np
import pandas as pd

# Audio / DSP
import librosa
import librosa.display
from scipy.signal import find_peaks

# Intentar usar Praat/parselmouth para formantes / HNR / jitter / shimmer
_HAS_PARSEL = False
try:
    import parselmouth
    _HAS_PARSEL = True
except Exception:
    _HAS_PARSEL = False
    warnings.warn("parselmouth no disponible: formantes/HNR/jitter/shimmer usarán aproximaciones.")

# ---------------------------
# Utils
# ---------------------------
def _basename_noext(path: str) -> str:
    return os.path.splitext(os.path.basename(path))[0]

def _safe_stats(x: np.ndarray) -> Dict[str, float]:
    x = np.asarray(x).astype(float)
    x = x[np.isfinite(x)]
    if x.size == 0:
        return {"mean": np.nan, "std": np.nan, "min": np.nan, "max": np.nan, "p25": np.nan, "p50": np.nan, "p75": np.nan}
    return {
        "mean": float(np.mean(x)),
        "std": float(np.std(x, ddof=1)) if x.size > 1 else 0.0,
        "min": float(np.min(x)),
        "max": float(np.max(x)),
        "p25": float(np.percentile(x, 25)),
        "p50": float(np.percentile(x, 50)),
        "p75": float(np.percentile(x, 75)),
    }

# ---------------------------
# 1) Carga de audio
# ---------------------------
def load_audio_mono(path: str, sr: int = 16000, trim_silence: bool = False) -> Dict[str, Any]:
    y, sr = librosa.load(path, sr=sr, mono=True)
    if trim_silence:
        yt, idx = librosa.effects.trim(y, top_db=30)
        if yt.size > 0:
            y = yt
    dur = len(y) / sr
    return {"y": y, "sr": sr, "duration": dur}

# ---------------------------
# 2) Métricas espectrales
# ---------------------------
def features_spectral(y: np.ndarray, sr: int) -> Dict[str, Any]:
    # STFT magnitud
    S = np.abs(librosa.stft(y, n_fft=1024, hop_length=256, win_length=1024))
    S_power = S**2

    # Centroide, ancho de banda, rolloff, flatness, contraste
    centroid = librosa.feature.spectral_centroid(S=S, sr=sr).ravel()
    bandwidth = librosa.feature.spectral_bandwidth(S=S, sr=sr).ravel()
    rolloff85 = librosa.feature.spectral_rolloff(S=S, sr=sr, roll_percent=0.85).ravel()
    flatness = librosa.feature.spectral_flatness(S=S).ravel()
    contrast = librosa.feature.spectral_contrast(S=S, sr=sr).T  # (frames, 7)

    # Flujo espectral (spectral flux): cambio frame a frame
    S_norm = S / (np.sum(S, axis=0, keepdims=True) + 1e-12)
    flux = np.sqrt(np.sum(np.diff(S_norm, axis=1)**2, axis=0))

    out: Dict[str, Any] = {}
    for name, vec in [
        ("spec_centroid", centroid),
        ("spec_bandwidth", bandwidth),
        ("spec_rolloff85", rolloff85),
        ("spec_flatness", flatness),
        ("spec_flux", flux),
    ]:
        st = _safe_stats(vec)
        for k, v in st.items():
            out[f"{name}_{k}"] = v

    # Contraste: promedios por banda (y su std)
    if contrast.size > 0:
        out.update({f"spec_contrast_band{b+1}_mean": float(np.mean(contrast[:, b])) for b in range(min(contrast.shape[1], 7))})
        out.update({f"spec_contrast_band{b+1}_std":  float(np.std(contrast[:, b], ddof=1)) for b in range(min(contrast.shape[1], 7))})
    else:
        for b in range(7):
            out[f"spec_contrast_band{b+1}_mean"] = np.nan
            out[f"spec_contrast_band{b+1}_std"] = np.nan

    # Energía por bandas (dominant band)
    freqs = librosa.fft_frequencies(sr=sr, n_fft=1024)
    bands = [(0,300),(300,800),(800,1500),(1500,3000),(3000,6000)]
    band_energy = []
    for (lo,hi) in bands:
        idx = (freqs>=lo)&(freqs<hi)
        band_energy.append(float(np.mean(S_power[idx,:])) if np.any(idx) else 0.0)
    out["dominant_band_idx"] = int(np.argmax(band_energy))  # 0..len(bands)-1
    out.update({f"band_energy_{lo}_{hi}_mean": e for (lo,hi),e in zip(bands, band_energy)})

    return out

# ---------------------------
# 3) MFCCs
# ---------------------------
def features_mfcc(y: np.ndarray, sr: int, n_mfcc: int = 13) -> Dict[str, Any]:
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc, n_fft=1024, hop_length=256)
    out: Dict[str, Any] = {}
    for i in range(n_mfcc):
        st = _safe_stats(mfcc[i, :])
        for k, v in st.items():
            out[f"mfcc{i+1}_{k}"] = v
    return out

# ---------------------------
# 4) Temporales (ZCR, RMS, tempo)
# ---------------------------
def features_temporal(y: np.ndarray, sr: int) -> Dict[str, Any]:
    zcr = librosa.feature.zero_crossing_rate(y, frame_length=1024, hop_length=256).ravel()
    rms = librosa.feature.rms(y=y, frame_length=1024, hop_length=256).ravel()

    # Tempo (aprox)
    try:
        tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
    except Exception:
        tempo = np.nan

    out: Dict[str, Any] = {"tempo_bpm": float(tempo) if np.isfinite(tempo) else np.nan}
    for name, vec in [("zcr", zcr), ("rms", rms)]:
        st = _safe_stats(vec)
        for k, v in st.items():
            out[f"{name}_{k}"] = v

    # Duración de segmentos energéticos (picos grandes ~ gritos / pausas)
    # Heurística: umbral en p75 de RMS
    if np.isfinite(out.get("rms_p75", np.nan)):
        thr = out["rms_p75"]
        active = rms > thr
        # Conteo de picos de energía (gritos)
        peaks, _ = find_peaks(rms, height=thr)
        out["energy_peak_count"] = int(len(peaks))
        out["energy_peak_rate_per_s"] = float(len(peaks) / (len(y)/sr)) if len(y)>0 else np.nan
        # Porcentaje del tiempo activo
        out["active_frames_ratio"] = float(np.mean(active))
    else:
        out["energy_peak_count"] = np.nan
        out["energy_peak_rate_per_s"] = np.nan
        out["active_frames_ratio"] = np.nan

    return out

# ---------------------------
# 5) Prosodia: Pitch, Jitter, Shimmer, Formantes, HNR
# ---------------------------
def _prosody_parselmouth(y: np.ndarray, sr: int) -> Dict[str, Any]:
    """Usar Praat si está disponible: F0, jitter, shimmer, HNR, formantes."""
    if not _HAS_PARSEL:
        return {}
    snd = parselmouth.Sound(y, sampling_frequency=sr)
    pitch = snd.to_pitch_cc(time_step=0.01, pitch_floor=60, pitch_ceiling=600)  # voz humana típica
    f0 = pitch.selected_array['frequency']
    f0 = f0[np.isfinite(f0) & (f0>0)]

    out: Dict[str, Any] = {}
    st = _safe_stats(f0)
    for k,v in st.items():
        out[f"f0_{k}"] = v
    out["voiced_ratio"] = float(len(f0) / (len(pitch.selected_array['frequency']) + 1e-12))

    # Jitter / Shimmer (Praat)
    # Jitter local (ppq5) y shimmer local (apq11) como medidas robustas
    point_process = parselmouth.praat.call(snd, "To PointProcess (periodic, cc)", 60, 600)
    try:
        jitter_local = parselmouth.praat.call([snd, point_process], "Get jitter (local)", 0, 0, 60, 600, 1.3, 1.6)
        jitter_ppq5 = parselmouth.praat.call([snd, point_process], "Get jitter (ppq5)", 0, 0, 60, 600, 1.3, 1.6)
        shimmer_local = parselmouth.praat.call([snd, point_process], "Get shimmer (local)", 0, 0, 60, 600, 1.3, 1.6, 1.6)
        shimmer_apq11 = parselmouth.praat.call([snd, point_process], "Get shimmer (apq11)", 0, 0, 60, 600, 1.3, 1.6, 1.6)
    except Exception:
        jitter_local=jitter_ppq5=shimmer_local=shimmer_apq11=np.nan

    out.update({
        "jitter_local": float(jitter_local) if np.isfinite(jitter_local) else np.nan,
        "jitter_ppq5": float(jitter_ppq5) if np.isfinite(jitter_ppq5) else np.nan,
        "shimmer_local": float(shimmer_local) if np.isfinite(shimmer_local) else np.nan,
        "shimmer_apq11": float(shimmer_apq11) if np.isfinite(shimmer_apq11) else np.nan,
    })

    # HNR (Harmonics-to-Noise Ratio)
    try:
        harmonicity = parselmouth.praat.call(snd, "To Harmonicity (cc)", 0.01, 60, 0.1, 1.0)
        hnr_mean = parselmouth.praat.call(harmonicity, "Get mean", 0, 0)
    except Exception:
        hnr_mean = np.nan
    out["hnr_mean_db"] = float(hnr_mean) if np.isfinite(hnr_mean) else np.nan

    # Formantes (F1, F2, F3) - mediana
    try:
        formant = parselmouth.praat.call(snd, "To Formant (burg)", 0.01, 5, 5500, 0.025, 50)
        # muestrear en 50 puntos
        t0, t1 = 0, snd.get_total_duration()
        ts = np.linspace(t0, t1, 50)
        F1, F2, F3 = [], [], []
        for t in ts:
            F1.append(parselmouth.praat.call(formant, "Get value at time", 1, t, 'Hertz', 'Linear'))
            F2.append(parselmouth.praat.call(formant, "Get value at time", 2, t, 'Hertz', 'Linear'))
            F3.append(parselmouth.praat.call(formant, "Get value at time", 3, t, 'Hertz', 'Linear'))
        for arr, name in [(F1, "F1"), (F2, "F2"), (F3, "F3")]:
            arr = np.array(arr, dtype=float)
            arr = arr[np.isfinite(arr) & (arr>0)]
            out[f"{name}_median_Hz"] = float(np.median(arr)) if arr.size else np.nan
    except Exception:
        out["F1_median_Hz"] = np.nan
        out["F2_median_Hz"] = np.nan
        out["F3_median_Hz"] = np.nan

    return out

def _prosody_approx(y: np.ndarray, sr: int) -> Dict[str, Any]:
    """Aproximaciones sin Praat: f0 con YIN/PYIN; jitter/shimmer aproximados."""
    out: Dict[str, Any] = {}

    # Pitch (f0)
    fmin, fmax = 60, 600
    try:
        f0 = librosa.pyin(y, fmin=fmin, fmax=fmax, frame_length=2048, hop_length=256)[0]
    except Exception:
        f0 = librosa.yin(y, fmin=fmin, fmax=fmax, frame_length=2048, hop_length=256)
    f0 = np.array(f0, dtype=float)
    f0_valid = f0[np.isfinite(f0) & (f0>0)]
    st = _safe_stats(f0_valid)
    for k,v in st.items():
        out[f"f0_{k}"] = v
    out["voiced_ratio"] = float(np.mean(np.isfinite(f0) & (f0>0))) if f0.size else np.nan

    # Jitter aprox: varianza relativa en periodo = var(diff(1/f0)) / mean(1/f0)
    if f0_valid.size > 5:
        T = 1.0 / f0_valid
        dT = np.diff(T)
        jitter_rel = (np.mean(np.abs(dT)) / (np.mean(T)+1e-12)) if dT.size else np.nan
        out["jitter_local"] = float(jitter_rel)
        out["jitter_ppq5"] = np.nan
    else:
        out["jitter_local"] = np.nan
        out["jitter_ppq5"] = np.nan

    # Shimmer aprox: cambio relativo de amplitud frame a frame en el envolvente RMS
    rms = librosa.feature.rms(y=y, frame_length=1024, hop_length=256).ravel()
    if rms.size > 5 and np.mean(rms) > 0:
        dr = np.diff(rms)
        shimmer_rel = np.mean(np.abs(dr)) / (np.mean(rms)+1e-12)
        out["shimmer_local"] = float(shimmer_rel)
        out["shimmer_apq11"] = np.nan
    else:
        out["shimmer_local"] = np.nan
        out["shimmer_apq11"] = np.nan

    # HNR aprox: relación energía "armónica" vs total con HPSS
    try:
        y_h, y_p = librosa.effects.hpss(y)
        hnr = 10*np.log10((np.mean(y_h**2)+1e-12) / (np.mean((y - y_h)**2)+1e-12))
    except Exception:
        hnr = np.nan
    out["hnr_mean_db"] = float(hnr) if np.isfinite(hnr) else np.nan

    # Formantes: sin Praat -> no fiable, devolvemos NaN
    out["F1_median_Hz"] = np.nan
    out["F2_median_Hz"] = np.nan
    out["F3_median_Hz"] = np.nan

    return out

def features_prosody(y: np.ndarray, sr: int) -> Dict[str, Any]:
    if _HAS_PARSEL:
        base = _prosody_parselmouth(y, sr)
        if base:
            return base
    return _prosody_approx(y, sr)

# ---------------------------
# 6) Rasgos específicos de llanto
# ---------------------------
def features_cry_specific(y: np.ndarray, sr: int) -> Dict[str, Any]:
    out: Dict[str, Any] = {}

    # Intensidad de gritos: picos de RMS altos
    rms = librosa.feature.rms(y=y, frame_length=1024, hop_length=256).ravel()
    thr = np.percentile(rms, 90) if rms.size else np.nan
    if np.isfinite(thr):
        peaks, props = find_peaks(rms, height=thr, distance=2)
        out["scream_peak_count"] = int(len(peaks))
        out["scream_rate_per_s"] = float(len(peaks) / (len(y)/sr)) if len(y)>0 else np.nan
        out["scream_intensity_p90"] = float(thr)
    else:
        out["scream_peak_count"] = np.nan
        out["scream_rate_per_s"] = np.nan
        out["scream_intensity_p90"] = np.nan

    # Patrones de respiración (inspiraciones cortas): valles profundos en RMS
    if rms.size:
        valleys, _ = find_peaks(-rms, distance=2)
        out["breath_valley_count"] = int(len(valleys))
        out["breath_rate_per_s"] = float(len(valleys) / (len(y)/sr)) if len(y)>0 else np.nan
    else:
        out["breath_valley_count"] = np.nan
        out["breath_rate_per_s"] = np.nan

    # Frecuencia dominante por frame y estadísticos
    S = np.abs(librosa.stft(y, n_fft=1024, hop_length=256))
    freqs = librosa.fft_frequencies(sr=sr, n_fft=1024)
    if S.size:
        dom_idx = np.argmax(S, axis=0)
        dom_freq = freqs[dom_idx]
        st = _safe_stats(dom_freq)
        for k,v in st.items():
            out[f"dominant_freq_{k}"] = v
    else:
        for k in ["mean","std","min","max","p25","p50","p75"]:
            out[f"dominant_freq_{k}"] = np.nan

    # Armonicidad: usamos HNR si está; si no, proxie = 1/flatness_mean
    # (se calculará en espectrales; si no, recomputamos aquí)
    try:
        flat = librosa.feature.spectral_flatness(y=y).ravel()
        out["harmonicity_proxy"] = float(1.0/ (np.mean(flat)+1e-12))
    except Exception:
        out["harmonicity_proxy"] = np.nan

    return out

# ---------------------------
# 7) Wrapper de UN archivo
# ---------------------------
def extract_features_from_file(path: str, sr: int = 16000, trim_silence: bool = False) -> Dict[str, Any]:
    loaded = load_audio_mono(path, sr=sr, trim_silence=trim_silence)
    y, sr, dur = loaded["y"], loaded["sr"], loaded["duration"]

    out: Dict[str, Any] = {
        "filepath": path,
        "filename": _basename_noext(path),
        "sr": int(sr),
        "duration_s": float(dur),
    }
    # Llamadas a subfunciones
    out.update(features_mfcc(y, sr))
    out.update(features_spectral(y, sr))
    out.update(features_temporal(y, sr))
    out.update(features_prosody(y, sr))
    out.update(features_cry_specific(y, sr))

    return out

# ---------------------------
# 8) Wrapper de DIRECTORIO
# ---------------------------
def process_wav_directory(
    dir_path: str,
    labels_df: Optional[pd.DataFrame] = None,
    label_key: str = "filename",
    label_col: str = "label",
    sr: int = 16000,
    trim_silence: bool = False,
    patterns: Optional[List[str]] = None
) -> pd.DataFrame:
    """
    - dir_path: carpeta con .wav
    - labels_df: opcional, DF con columnas [label_key, label_col] para unir etiquetas
    - label_key: columna en labels_df que coincide con 'filename' (sin extensión)
    - label_col: nombre de la columna de etiqueta a anexar
    - sr: frecuencia de muestreo para librosa.load
    - trim_silence: recorte inicial/final por silencio (top_db=30)
    - patterns: lista de patrones glob adicionales (por defecto ['*.wav','*.WAV'])
    """
    if patterns is None:
        patterns = ["*.wav", "*.WAV"]

    file_list: List[str] = []
    for p in patterns:
        file_list.extend(glob.glob(os.path.join(dir_path, p)))
    file_list = sorted(set(file_list))

    rows: List[Dict[str, Any]] = []
    for fp in file_list:
        try:
            feats = extract_features_from_file(fp, sr=sr, trim_silence=trim_silence)
            rows.append(feats)
        except Exception as e:
            warnings.warn(f"Error procesando {fp}: {e}")

    df = pd.DataFrame(rows)

    # Unir labels si se proporcionan
    if labels_df is not None and not labels_df.empty:
        if label_key not in labels_df.columns:
            raise ValueError(f"labels_df no contiene la columna '{label_key}'")
        if label_col not in labels_df.columns:
            raise ValueError(f"labels_df no contiene la columna de etiqueta '{label_col}'")
        # Normalizar nombre base sin extensión para unir
        labels_tmp = labels_df.copy()
        labels_tmp = labels_tmp.dropna(subset=[label_key])
        labels_tmp[label_key] = labels_tmp[label_key].astype(str).apply(lambda s: os.path.splitext(os.path.basename(s))[0])
        df = df.merge(labels_tmp[[label_key, label_col]], left_on="filename", right_on=label_key, how="left")
        df = df.drop(columns=[label_key])

    return df


In [ ]:


# 2) Procesar una carpeta con wavs
df_feats = process_wav_directory(
    dir_path="data/raw/nuria_wavs",
    labels_df=labels,       # o None si no tienes etiquetas
    label_key="nombre_archivo",   # la columna en 'labels' que coincide con el nombre de archivo sin extensión
    label_col="clase",      # cómo se llama tu etiqueta
    sr=16000,
    trim_silence=False
)

# 3) Guardar
df_feats.to_parquet("data/processed/features_wavs.parquet", index=False)
# o
df_feats.to_csv("data/processed/features_wavs.csv", index=False)


C:\Users\narro\AppData\Local\Temp\ipykernel_2392\923343218.py:134: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  out: Dict[str, Any] = {"tempo_bpm": float(tempo) if np.isfinite(tempo) else np.nan}


In [ ]:
df_feats

,filepath,filename,sr,duration_s,mfcc1_mean,mfcc1_std,mfcc1_min,mfcc1_max,mfcc1_p25,mfcc1_p50,...,breath_rate_per_s,dominant_freq_mean,dominant_freq_std,dominant_freq_min,dominant_freq_max,dominant_freq_p25,dominant_freq_p50,dominant_freq_p75,harmonicity_proxy,clase
0,data/raw/nuria_wavs\1.wav,1,16000,225.049813,-260.504979,31.190871,-631.306702,-154.154648,-286.525536,-264.805817,...,9.446797,506.843852,586.136456,15.625,7265.625,31.25000,62.5000,906.25000,85.467972,1
1,data/raw/nuria_wavs\100.wav,100,16000,5.796875,-178.781611,95.958995,-553.396790,15.802969,-202.739059,-181.572449,...,7.762803,777.117769,764.966971,0.000,3250.000,0.00000,859.3750,1015.62500,20780.015625,4
2,data/raw/nuria_wavs\101.wav,101,16000,29.761563,-137.511060,83.621335,-488.558655,28.949188,-193.400116,-163.317337,...,7.896091,1029.444855,973.150165,0.000,3500.000,0.00000,953.1250,1468.75000,17780.593750,4
3,data/raw/nuria_wavs\102.wav,102,16000,34.069125,-132.816667,69.872853,-485.048492,-1.397134,-180.116970,-140.632202,...,6.839037,1535.871479,1169.971380,0.000,3546.875,796.87500,1234.3750,2937.50000,37242.707031,4
4,data/raw/nuria_wavs\103.wav,103,16000,25.308812,-122.601546,85.305969,-500.170227,26.472803,-185.027626,-117.398560,...,6.954099,1774.622709,1212.267921,0.000,3500.000,894.53125,1835.9375,2984.37500,6447.050293,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,data/raw/nuria_wavs\95.wav,95,16000,18.815875,-88.359868,108.187242,-486.380524,29.091347,-154.860275,-45.441927,...,8.131432,1396.537521,811.244249,0.000,3437.500,968.75000,1468.7500,1859.37500,41133.957031,4
127,data/raw/nuria_wavs\96.wav,96,16000,14.375625,-118.691954,93.280294,-501.817322,18.415400,-188.839500,-98.390236,...,7.860528,1196.468298,844.754051,0.000,3390.625,757.81250,1250.0000,1781.25000,18403.550781,4
128,data/raw/nuria_wavs\97.wav,97,16000,23.477750,-119.710004,90.723809,-491.256378,12.181683,-192.161549,-104.274063,...,8.050175,1143.786189,784.429975,0.000,3437.500,0.00000,1250.0000,1781.25000,34396.402344,4
129,data/raw/nuria_wavs\98.wav,98,16000,15.636500,-116.078857,89.180134,-482.326141,49.768780,-182.262482,-130.503113,...,8.441787,1015.257541,767.797833,0.000,3265.625,0.00000,1031.2500,1417.96875,25186.023438,4


In [10]:

results = pd.read_csv('data/raw/nuria_wavs_test/results.csv')

df_feats_test = process_wav_directory(
    dir_path="data/raw/nuria_wavs_test",
    labels_df=results,       # o None si no tienes etiquetas
    label_key="nombre_archivo",   # la columna en 'labels' que coincide con el nombre de archivo sin extensión
    label_col="clase",      # cómo se llama tu etiqueta
    sr=16000,
    trim_silence=False
)

# 3) Guardar
df_feats.to_parquet("data/processed/features_wavs_train_test.parquet", index=False)
# o
df_feats.to_csv("data/processed/features_wavs_train_test.csv", index=False)

C:\Users\narro\AppData\Local\Temp\ipykernel_2392\923343218.py:134: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  out: Dict[str, Any] = {"tempo_bpm": float(tempo) if np.isfinite(tempo) else np.nan}
